In [1]:
import torch
from torch import nn

In [2]:
batch_size, C, H, W = 11, 3, 16, 14
K, padding = 5, 2

In [6]:
v_pj = nn.Linear(C, C)
attn = nn.Linear(C, K ** 4)
unfold = nn.Unfold(K, padding=padding)
fold = nn.Fold(output_size=(H, W), kernel_size=K, padding=padding)

In [18]:
x = torch.randn(batch_size, H, W, C)
print(x.shape)
v = v_pj(x).permute(0, 3, 1, 2)
print(v.shape)
v = unfold(v)
print(v.shape)
v = v.reshape(batch_size, C, K * K, H * W).permute(0, 3, 2, 1)
print(v.shape)

a = attn(x)
print(a.shape)
a = a.reshape(batch_size, H * W, K * K, K * K).softmax(dim=-1)

print('----')
print(a.shape, v.shape)
x = torch.einsum('bljk, blkc -> bljc', a, v)
print(x.shape)
x = x.permute(0, 3, 2, 1)
print(x.shape)
x = x.reshape(batch_size, C * K * K, H * W)
x = fold(x)
print(x.shape)
x = x.permute(0, 2, 3, 1)
print(x.shape)

torch.Size([11, 16, 14, 3])
torch.Size([11, 3, 16, 14])
torch.Size([11, 75, 224])
torch.Size([11, 224, 25, 3])
torch.Size([11, 16, 14, 625])
----
torch.Size([11, 224, 25, 25]) torch.Size([11, 224, 25, 3])
torch.Size([11, 224, 25, 3])
torch.Size([11, 3, 25, 224])
torch.Size([11, 3, 16, 14])
torch.Size([11, 16, 14, 3])
